In [ ]:
#import pipeline from transformers module
import pandas as pd
from transformers import pipeline

# Load the BART-large MNLI model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Load the conversations from the CSV file
df1 = pd.read_csv("/Call Centre Design 1.csv")

# Define the labels for financial investments or the financial/investment/insurance industry
candidate_labels = ['Asset', 'Bond', 'Brokerage', 'Card', 'CDO', 'CLO', 'Collectibles', 'Commodities', 'Credit', 'Cryptocurrency', 'Debt', 'Derivatives', 'Digital', 'ETF', 'Estate', 'Exchange', 'Finance', 'Financial', 'Foreign', 'Fund', 'Gold', 'Green', 'Hedge', 'High-Frequency', 'Infrastructure', 'Insurance', 'Investment', 'IPO', 'Lending', 'Limited', 'Loan', 'Management', 'Master', 'MBS', 'Merchant', 'Municipal', 'Mutual', 'Online', 'Open-End', 'Options', 'Payment', 'Pension', 'Platinum', 'Portfolio', 'Precious', 'Private', 'Processing', 'Real', 'REIT', 'Retirement', 'Reverse', 'Risk', 'Savings', 'Secondary', 'Securities', 'Silver', 'Sovereign', 'Split', 'Structured', 'Tax', 'Trading', 'Treasury', 'Venture', 'Wealth', 'Wine', 'Actuarial', 'Algorithmic', 'Commodities', 'Commercial', 'Employee', 'ESOP', 'Futures', 'Municipal', 'Quantitative', 'Risk']


# Create a list to store the probability scores for each conversation
scores = []

# Iterate over the conversations and calculate the probability score for each one
for conversation in df1[" Comprehensive Transcript"]:
    print(conversation)
    result = classifier(conversation, candidate_labels)
    print(result["scores"][0])
    scores.append(result["scores"][0])

# Add the scores to the conversations dataframe
df1["importance_score"] = scores

# Save the updated conversations dataframe to a new CSV file
df1.to_csv("/Call_Centre_Design_1_with_scores.csv", index=False)


okay <mandarin>来来:lai lai</mandarin> okay <mandarin>来:lai</mandarin> round two (ppl) <S> <Z> <Z> <Z> <S> <S> <Z> <Z> <S> <Z> <S> <S> <Z> <S> <Z> okay alright so (err) miss lily [ah] thank you again [ah] for dropping this call to let us know so (uh) my name is jack over here and nice talking to you so again I think I hear few fair bit of request from your side [ah] in terms of the call enquiry so maybe I can (err) (ppo) check with you a bit more understand a bit from more from you [ah] because I think over here is (uh) (ppo) (uh) you've mentioned that you'll be travelling in (err) family in a tour so (err) there will be I think apart from you your husband and your children there will also be these two elderly [ah] which happens to be your parent so over here is (uh) for your father right who you mentioned is wheelchair bound right (uh) just like to understand a bit more [ah] like could you share with me like (err) what's his like current age and also things like (err) for his wheelchair

KeyboardInterrupt: ignored